# Data Collection and Preprocessing

Importing Libraries

In [2]:
from nba_api.stats.endpoints import leaguedashteamstats
import pandas as pd

Loading in Data (Looking at one year of data looks like)

In [3]:
data =  leaguedashteamstats.LeagueDashTeamStats(season='2023-24',measure_type_detailed_defense='Advanced')#Focusing more on advanced stats
datadf = data.get_data_frames()[0]
datadf.head()


,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,...,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK
0,1610612737,Atlanta Hawks,82,36,46,0.439,3971.0,114.0,116.4,116.7,...,18,21,5,16,12,13,17,18,6,22
1,1610612738,Boston Celtics,82,64,18,0.780,3966.0,120.2,122.2,109.0,...,3,13,14,7,5,1,2,1,19,1
2,1610612751,Brooklyn Nets,82,32,50,0.390,3961.0,110.4,112.4,113.1,...,19,19,12,19,20,14,25,27,25,21
3,1610612766,Charlotte Hornets,82,21,61,0.256,3946.0,107.1,108.6,117.0,...,25,22,27,22,29,21,26,28,22,29
4,1610612741,Chicago Bulls,82,39,43,0.476,3996.0,112.4,114.0,113.2,...,13,24,11,17,14,3,22,21,28,20


Creating Dataframe of Team Data from the last 5 years(Not including first year of COVID when teams played different amounts of games)

In [4]:
seasons = ['2023-24','2022-23','2021-22','2020-21','2018-19']

dfs = []
for season in seasons:
    stats = leaguedashteamstats.LeagueDashTeamStats(season=season,measure_type_detailed_defense='Advanced')
    statsdf = stats.get_data_frames()[0]
    statsdf.insert(loc = 0 , column = 'SEASON', value=season)
    dfs.append(statsdf)

NBA_df = pd.concat(dfs, ignore_index= True)
NBA_df

,SEASON,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,E_OFF_RATING,OFF_RATING,...,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK
0,2023-24,1610612737,Atlanta Hawks,82,36,46,0.439,3971.0,114.0,116.4,...,18,21,5,16,12,13,17,18,6,22
1,2023-24,1610612738,Boston Celtics,82,64,18,0.780,3966.0,120.2,122.2,...,3,13,14,7,5,1,2,1,19,1
2,2023-24,1610612751,Brooklyn Nets,82,32,50,0.390,3961.0,110.4,112.4,...,19,19,12,19,20,14,25,27,25,21
3,2023-24,1610612766,Charlotte Hornets,82,21,61,0.256,3946.0,107.1,108.6,...,25,22,27,22,29,21,26,28,22,29
4,2023-24,1610612741,Chicago Bulls,82,39,43,0.476,3996.0,112.4,114.0,...,13,24,11,17,14,3,22,21,28,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2018-19,1610612758,Sacramento Kings,82,39,43,0.476,3946.0,108.3,109.6,...,7,15,18,26,24,4,16,20,3,18
146,2018-19,1610612759,San Antonio Spurs,82,48,34,0.585,3961.0,111.1,112.2,...,4,12,29,6,16,1,6,8,22,10
147,2018-19,1610612761,Toronto Raptors,82,58,24,0.707,3976.0,110.9,112.6,...,13,11,19,18,13,14,3,4,15,4
148,2018-19,1610612762,Utah Jazz,82,50,32,0.610,3951.0,108.8,110.3,...,15,8,12,1,3,27,5,7,14,3
